## Классификация твитов по тональности

Вначале очистим наш текст. Уберем стоп слова, цыфры, символы. Лемматизацию делать не будем, так как как для получения embedding-а будем использовать BPE.

In [1]:
import regex as re
from tqdm import tqdm
from scipy import sparse
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
columns = ['id', 'date', 'name', 'text', 'type', 'rep', 'rtw', 'fav', 'stcount', 
            'fol', 'frien', 'listcount']

In [3]:
positive = pd.read_csv('Data/positive.csv', sep=';',  names=columns)
negative = pd.read_csv('Data/negative.csv', sep=';',  names=columns)

In [4]:
positive['is_positive'] = 1
negative['is_positive'] = 0

In [5]:
negative.head(3)

,id,date,name,text,type,rep,rtw,fav,stcount,fol,frien,listcount,is_positive
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2,0
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0,0


In [6]:
df = (positive.append(negative)).reset_index(drop=True)

In [7]:
positive.shape, negative.shape, df.shape

((114911, 13), (111923, 13), (226834, 13))

In [8]:
df = pd.DataFrame(df[['text', 'is_positive']])

In [9]:
word = re.compile('[А-Яа-я]+')
df['text'] = df['text'].apply(lambda x: ' '.join(re.findall(word, x.lower())))

In [10]:
df.head()

,text,is_positive
0,хоть я и школота но поверь у нас то же самое о...,1
1,да все таки он немного похож на него но мой ма...,1
2,ну ты идиотка я испугалась за тебя,1
3,кто то в углу сидит и погибает от голода а мы ...,1
4,вот что значит страшилка но блин посмотрев все...,1


In [11]:
from nltk.corpus import stopwords
from bpemb import BPEmb
from pathlib import WindowsPath

In [17]:
bpemb_ru = BPEmb(lang="ru", dim=100, cache_dir = WindowsPath('C:\My Programs'))

In [19]:
df = (positive.append(negative)).reset_index(drop=True)

In [20]:
df = pd.DataFrame(df[['text', 'is_positive']])

In [21]:
word = re.compile('[А-Яа-я]+')
df['text'] = df['text'].apply(lambda x: ' '.join(re.findall(word, x.lower())))

In [22]:
df.head()

,text,is_positive
0,хоть я и школота но поверь у нас то же самое о...,1
1,да все таки он немного похож на него но мой ма...,1
2,ну ты идиотка я испугалась за тебя,1
3,кто то в углу сидит и погибает от голода а мы ...,1
4,вот что значит страшилка но блин посмотрев все...,1


In [23]:
stopw = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также']
def  stop_words(text, stopwords = stopw):
    try:
        return " ".join([token for token in text.split() if not token in stopwords and len(token) > 2])
    except:
        return ""

In [24]:
df['text'] = df['text'].apply(lambda x: stop_words(x))

Теперь получим сами embedding для слов, усредним их на каждом предложении и отдадим в качестве признаков

In [23]:
def sent_to_bpe(sent):
    vec = bpemb_ru.embed(sent)
    if len(vec):
        return(np.mean(vec, axis=0))
    else:
        return(np.zeros(100))

In [24]:
df['text_bpe'] = df['text'].apply(lambda x: sent_to_bpe(x))

In [25]:
df.head()

,text,is_positive,text_bpe
0,школота поверь самое общество профилирующий пр...,1,"[0.014792068, -0.10914229, 0.094901, 0.0275122..."
1,таки немного похож мальчик равно,1,"[-0.337034, -0.063679755, 0.1094545, 0.0075524..."
2,идиотка испугалась,1,"[-0.17493764, 0.105021626, 0.22462825, 0.23207..."
3,углу сидит погибает голода порции взяли хотя ж...,1,"[-0.30528587, 0.10548739, 0.0524021, 0.1353126..."
4,значит страшилка блин посмотрев части создастс...,1,"[-0.19662201, -0.011566614, -0.03991857, 0.264..."


Отделим 25% положительных и 25% отрицательных твитов как тестовую выборку. И попробуем классифицироватьнаши твиты с помощью нейросети

In [27]:
splitter = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 13)

for train_index, test_index in splitter.split(df, df['is_positive']):
    X_train = df.iloc[train_index]
    X_test = df.iloc[test_index]
    y_train = df['is_positive'].iloc[train_index]
    y_test = df['is_positive'].iloc[test_index]

In [28]:
X_train = X_train['text_bpe'].tolist()
X_test = X_test['text_bpe'].tolist()

In [50]:
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import Input
from keras.activations import tanh, sigmoid
from keras.models import Model
import keras

In [85]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [72]:
X_train.shape

(170125, 100)

In [74]:
nn_input = Input(shape=(100,))
x = Dense(256, activation=tanh)(nn_input)
x = BatchNormalization()(x)
x = Dense(256, activation=tanh)(x)
x = BatchNormalization()(x)
x = Dense(128, activation=tanh)(x)
x = BatchNormalization()(x)
output = Dense(1, activation=sigmoid)(x)

model = Model(inputs=[nn_input], output=[output] )

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  # Remove the CWD from sys.path while we load stuff.


In [119]:
model.evaluate

optimizer = keras.optimizers.Adam(lr=4e-3)
model.compile(optimizer=optimizer, 
              loss='binary_crossentropy',  # функция потерь binary_crossentropy (log loss)
              metrics=['accuracy'])

In [120]:
model.fit(X_train, y_train, batch_size=200000, epochs=100)

Epoch 1/100
170125/170125 [==============================] - 5s 31us/step - loss: 0.5717 - acc: 0.6967
Epoch 2/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.6243 - acc: 0.6438
Epoch 3/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.5753 - acc: 0.6934
Epoch 4/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.6007 - acc: 0.6675
Epoch 5/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5947 - acc: 0.6718
Epoch 6/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5782 - acc: 0.6899
Epoch 7/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.5785 - acc: 0.6882
Epoch 8/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5851 - acc: 0.6795
Epoch 9/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5830 - acc: 0.6833
Epoch 10/100
170125/170125 [==============================] - 3s 18us/ste

Epoch 80/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5406 - acc: 0.7230
Epoch 81/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5400 - acc: 0.7238
Epoch 82/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5394 - acc: 0.7243
Epoch 83/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5388 - acc: 0.7247
Epoch 84/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5381 - acc: 0.7251
Epoch 85/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5375 - acc: 0.7257
Epoch 86/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.5369 - acc: 0.7262
Epoch 87/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5363 - acc: 0.7268
Epoch 88/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.5356 - acc: 0.7273
Epoch 89/100
170125/170125 [==============================] - 4s

In [121]:
y_pred = model.predict(X_test)

In [122]:
y_pred[y_pred<0.5] = int(0)
y_pred[y_pred>0.5] = int(1)

In [123]:
accuracy_score(y_test, y_pred)

0.601033345677053

Результат не очень хороший и похоже наша сеть немного переобучилась. Но настраивать её сейчас не будем. Вместо этого теперь будем не усреднять эмбединги, а подадим их в рекурентную сеть

In [25]:
from torch.nn.modules import LSTM
import torch.nn as nn
import torch.optim as optim
import torch

In [27]:
df['text_bpe'] = df['text'].apply(lambda x: sent_to_bpe_2(x))

In [244]:
df['opt_num'] = df['text_bpe'].apply(lambda x: x.shape[0])
df.opt_num.describe(percentiles=[0.7, 0.8, 0.9, 0.95])

count    226834.000000
mean         14.117121
std           8.225647
min           1.000000
50%          13.000000
70%          17.000000
80%          20.000000
90%          24.000000
95%          27.000000
max         111.000000
Name: opt_num, dtype: float64

In [26]:
def sent_to_bpe_full(sent):
    vec = bpemb_ru.embed(sent)
    if len(vec):
        return(vec)
    else:
        return(np.zeros(100).reshape(1,-1))

In [27]:
df['text_bpe'] = df['text'].apply(lambda x: sent_to_bpe_full(x))

In [28]:
splitter = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 13)

for train_index, test_index in splitter.split(df, df['is_positive']):
    X_train = df.iloc[train_index]
    X_test = df.iloc[test_index]
    y_train = df['is_positive'].iloc[train_index]
    y_test = df['is_positive'].iloc[test_index]

In [29]:
X_train = X_train['text_bpe'].tolist()
X_test = X_test['text_bpe'].tolist()

In [57]:
def GetValues(words):
    return bpemb_ru.encode_ids(words)
def GetWords(values):
    return bpemb_ru.decode_ids(values)

In [106]:
from torch.utils.data import Dataset, DataLoader

class BpeDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.vocab_size = bpemb_ru.vocab_size
        self.data = X_train
        self.target = y_train
        self.eos = bpemb_ru.vectors[bpemb_ru.EOS].tolist()
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target.iloc[index]
        x_list = x[1:].tolist()
        x_list.append(self.eos)
        y_for_seq2seq = np.array(x_list)
        return x, y     
        
    def __len__(self):
        return len(self.data)

In [107]:
trn_ds = BpeDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [108]:
class LSTMmodel(nn.Module):
    def __init__(self, input_size=100, hidden_size=50, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.out = nn.Linear(hidden_size, bpemb_ru.vocab_size)
        self.vocab_size = bpemb_ru.vocab_size
        self.softmax = nn.LogSoftmax(dim=1)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.Tanh(),
            nn.Dropout(p=0.3),
            nn.Linear(256, 128),
            nn.Tanh(),
            nn.Dropout(p=0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        self.hidden_size = hidden_size
        
    def forward(self, x, c_prev, h_prev):
        x = x.float()
        lstm_out, (c_prev, h_prev) = self.lstm(x, (c_prev, h_prev))
        tag_space = self.classifier(lstm_out.view(len(x), -1))
        return tag_space, (c_prev, h_prev)

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [109]:
hidden_size = 50
model = LSTMmodel()

In [110]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        optimizer.zero_grad()
        c_prev = torch.zeros([1, 1, hidden_size], dtype=torch.float, device='cpu')
        h_prev = torch.zeros_like(c_prev)
        x, y = x.to("cpu"), y.to("cpu")
        for i in range(x.shape[1]):
            y_pred, (c_prev, h_prev)  = model.forward(x[:, i].view(1, 1, -1), c_prev, h_prev)
            y_true = torch.tensor(y.view(1, -1)).float()
        loss = loss_fn(y_pred, y_true)
    
         
        if loss != 0:
            loss.backward()
            optimizer.step()
        if line_num % 10000 == 0:
            print (line_num)

In [111]:
def train(model, loss_fn, optimizer, epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer)

In [112]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [162]:
train(model, loss_fn, optimizer, epochs = 20)

In [ ]:
torch.save(model, "rnn.model")

In [117]:
the_model = torch.load("rnn.model")

In [122]:
class BpeDataset_test(Dataset):
    def __init__(self):
        super().__init__()
        self.data = X_test
        
    def __getitem__(self, index):
        x = self.data[index]
        return x    
        
    def __len__(self):
        return len(self.data)

In [123]:
trn_ds_test = BpeDataset_test()
trn_dl_test = DataLoader(trn_ds_test, shuffle=True)

In [125]:
def model_fit(model):
    list_predict = []
    for line_num, x in enumerate(trn_dl_test):
        c_prev = torch.zeros([1, 1, hidden_size], dtype=torch.float, device='cpu')
        h_prev = torch.zeros_like(c_prev)
        x = x.to("cpu")
        for i in range(x.shape[1]):
            y_pred, (c_prev, h_prev)  = model.forward(x[:, i].view(1, 1, -1), c_prev, h_prev)
        list_predict.append(y_pred)
    return list_predict

In [142]:
y_pred = model_fit(model)

In [157]:
list_pred = []
for i in y_pred:
    list_pred.append(1 if float(i.view(-1)) > 0.5 else 0)

In [179]:
accuracy_score(y_test, list_pred)

0.6722742421837804

Видим что результат на RNN получается лучше. Так же для более качественного предсказания нужно реализовать attention